In [6]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
from datetime import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import pandas as pd
import numpy as np
import glob
import nbimporter
import gzip

In [7]:
# Define the path to the saved .pickle.gz file
file_path = "C:\\Users\\SamuliMustonen\\Documents\\Ready Solutions\\Docs\\StockTrading\\Data\\rawComplete\\all_raw_data_2022-01-01_to_2024-09-30.pickle.gz"

# Load the DataFrame from the compressed pickle file
try:
    df = pd.read_pickle(file_path, compression='gzip')
    print("File loaded successfully!")
    print(df.head())  # Display the first few rows
except Exception as e:
    print(f"Error loading file: {e}")


File loaded successfully!
  symbol  close   open   high    low   volume   timestamp  \
0   AACT  10.12  10.10  10.15  10.10   603100  2023-06-12   
1   AACT  10.11  10.14  10.14  10.11     5500  2023-06-13   
2   AACT  10.10  10.11  10.13  10.10   143900  2023-06-14   
3   AACT  10.11  10.11  10.11  10.10  2061100  2023-06-15   
4   AACT  10.11  10.12  10.12  10.11   251000  2023-06-16   

   exceptionalCandleSize  ema20  ema50  openHigher  averageVolume  \
0                      0  10.12  10.12           0            NaN   
1                      0  10.12  10.12           0            NaN   
2                      0  10.12  10.12           0            NaN   
3                      0  10.12  10.12           0            NaN   
4                      0  10.12  10.12           0            NaN   

   strongVolume  strongVolume6MoPrior  accVolume  uptrend  closeHigh  pegAlert  
0             0                     0          0        0          0         0  
1             0               

In [8]:
# Check for missing values in the dataset
missing_data = df.isnull().sum()
print(missing_data)

symbol                        0
close                         0
open                          0
high                          0
low                           0
volume                        0
timestamp                     0
exceptionalCandleSize         0
ema20                         0
ema50                         0
openHigher                    0
averageVolume            113875
strongVolume                  0
strongVolume6MoPrior          0
accVolume                     0
uptrend                       0
closeHigh                     0
pegAlert                      0
dtype: int64


In [9]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


In [4]:
# Get summary statistics
print(df.describe())

               close           open           high            low  \
count  742709.000000  742709.000000  742709.000000  742709.000000   
mean       25.280034      25.294837      25.746279      24.836079   
std        44.232542      44.286322      44.986244      43.605948   
min         0.250000       0.250000       0.250000       0.130000   
25%         8.860000       8.870000       9.050000       8.670000   
50%        15.430000      15.440000      15.720000      15.160000   
75%        28.820000      28.830000      29.370000      28.290000   
max      1743.820000    1900.000000    2555.300000    1730.670000   

             volume  exceptionalCandleSize          ema20          ema50  \
count  7.427090e+05          742709.000000  742709.000000  742709.000000   
mean   8.446777e+05               0.034748      25.407491      25.616118   
std    2.944342e+06               0.183142      44.529832      45.078963   
min    0.000000e+00               0.000000       0.420000       0.600000  

In [10]:
# List all unique symbols in the 'symbol' column
unique_symbols = df['symbol'].unique()
print(unique_symbols)

['AACT' 'AAOI' 'AAT' ... 'ZKH' 'ZUO' 'ZYME']


In [17]:
# Function to filter date range and symbols that are alerted by PEG model.
def model_peg_list(data):
    # symbol_filter = 'ABUS'  # Replace with the symbol you want to filter
    date_filter = (data['timestamp'] >= '2022-01-01') & (data['timestamp'] <= '2024-09-30')
    pegAlert = (data['pegAlert']==1)

     # Apply filters and select unique values from the 'symbol' column
    modelPEGlist = data[date_filter & pegAlert]['symbol'].unique()
    
    return modelPEGlist

In [33]:
# Filter and sort the original DataFrame using the list of unique symbols
def filter_data_by_symbols(data, symbols_list):
    # Filter the DataFrame to include only rows where 'symbol' is in the symbols_list
    filtered_df = data[data['symbol'].isin(symbols_list)]

    # Sort the filtered DataFrame by 'symbol' and 'timestamp' (or another column if needed)
    sorted_df = filtered_df.sort_values(by=['timestamp'])

    return sorted_df

In [34]:
# Example usage
pegList = model_peg_list(df)  # List of unique symbols
sorted_df = filter_data_by_symbols(df, pegList)

# Display the sorted DataFrame
# print(sorted_df)

In [35]:
# Main function to run the process
def main():
    
    pegList = model_peg_list(df)  # List of unique symbols
    sorted_df = filter_data_by_symbols(df, pegList) # Data filtered by unique symbols


if __name__ == "__main__":
    main()

In [ ]:
# Filter for a specific symbol and date range within 2023
# symbol_filter = 'AGL'  # Replace with the symbol you want to filter
# date_filter = (df['timestamp'] >= '2023-01-01') & (df['timestamp'] <= '2023-12-31')
# openHigher = (df['openHigher'] == 1)
# accVolume = (df['accVolume']==1)
# uptrend = (df['uptrend']==1)
# excCandle = (df['exceptionalCandleSize']==1)
# closeHigh = (df['closeHigh']==1)

# # Apply all filters at once
# filtered_df = df[
#     # (df['symbol'] == symbol_filter) & 
#     date_filter & 
#     openHigher & 
#     accVolume & 
#     uptrend & 
#     excCandle &
#     closeHigh
# ]

# # View the filtered DataFrame
# print(filtered_df)

In [36]:
# Save the alert list to csv.
start_date = "2022-01-01"  
end_date = "2024-09-30"    
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')  # current timestamp

save_path = "C:\\Users\\SamuliMustonen\\Documents\\Ready Solutions\\Docs\\StockTrading\\Data\\modelPEG"
file_name = f"model_PEG_alert_list_{start_date}_to_{end_date}_{timestamp}.csv"
full_path = f"{save_path}\\{file_name}"

# Save the DataFrame to CSV
sorted_df.to_csv(full_path, index=False)

print(f"File saved to: {full_path}")

File saved to: C:\Users\SamuliMustonen\Documents\Ready Solutions\Docs\StockTrading\Data\modelPEG\model_PEG_alert_list_2022-01-01_to_2024-09-30_20241020_220028.csv
